In [1]:
import warnings
warnings.simplefilter("ignore")
from sys import exit
import os.path
import cv2
import numpy as np
import pandas as pd
from tkinter import Tk
from tkinter.filedialog import askopenfilename
from tkinter.filedialog import askdirectory
from scipy import ndimage
from skimage import measure, io, exposure
from pathlib import Path
from skimage.feature import peak_local_max
import csv

ModuleNotFoundError: No module named 'cv2'

In [ ]:
imgname = "E:/Downloads/use.tif"
img = io.imread(imgname,-1)


In [3]:
'''
outL: box size when identifying molecules
splitter: amount of frames between check-ins to get new molecules
frameRate = frame rate of camera to determine time
'''
outL = int(99)
splitter = 50
frameRate = 10

'''
showimgs: shows all images for testing, set as false for just running
saveimgs: allows user to save images
saveForTest: saves additional files full of images
saveForPeaks: saves additional files for precise peak reading
'''
showimgs = False
saveimgs = True
saveForTest = False
saveForPeaks = False

enter = 13
right = 83
left = 81
esc = 27

wantGain = False
'''
Define pixel size of current camera in nm/pix-el: MD: 114, Andor: 128, Sam: 101.4
'''
pixelSize = 128
refPt = []

In [4]:
def my_convert_16U_2_8U(image):
    min_ = np.amin(image)
    max_ = np.amax(image)
    a = 255/float(max_-min_)
    b = -a*min_ 
    img8U = np.zeros(image.shape,np.uint8)
    cv2.convertScaleAbs(image,img8U,a,b)
    return img8U

In [5]:
def getBrightBox(img, point, outLength):
    shap = img.shape
    startx = int(max(0, point[1] - outLength))
    starty = int(max(0, point[0] - outLength))
    endx = int(min(shap[1], point[1] + outLength+1))
    endy = int(min(shap[0], point[0] + outLength+1))
    ml = int(min(0, point[1] - outLength))
    mr = int(max(0, point[1] + outLength + 1 - shap[1]))
    mt = int(min(0, point[0] - outLength))
    mb = int(max(0, point[0] + outLength + 1 - shap[0]))
    miss = [int(mt*-1), int(mb), int(ml*-1), int(mr)]
    brightBox = img[starty:endy,startx:endx]
    avg = brightBox.mean()
    return(brightBox, startx, starty, avg, miss)

In [6]:
def makeFolders(numPoints,dirtoo):
    for i in range(numPoints[0],numPoints[1]):
        folderr = os.path.join(dirtoo, str(i).zfill(3))
        lanalysis = os.path.join(folderr, 'analysis.txt')
        if os.path.exists(folderr) == False:
            os.mkdir(folderr)
        into = open(lanalysis, 'w')
        into.write(header)
        into.close()
        if saveimgs == True:
            folderb = os.path.join(folderr, 'brightBoxes')
            foldero = os.path.join(folderr, 'edgeDetection')
            folderf = os.path.join(folderr, 'captures')
            foldertm = os.path.join(folderr, 'trackedMolecule')
            folderp = os.path.join(folderr, 'peaks')
            folderpl = os.path.join(folderr, 'plots')
            if os.path.exists(folderb) == False:
                os.mkdir(folderb)
                os.mkdir(foldero)
                os.mkdir(folderf)
                os.mkdir(foldertm)

In [7]:
def makeFolder(i,dirtoo):
    folderr = os.path.join(dirtoo, str(i).zfill(3))
    if os.path.exists(folderr) == False:
        os.mkdir(folderr)
    if saveimgs == True:
        folderb = os.path.join(folderr, 'brightBoxes')
        foldero = os.path.join(folderr, 'edgeDetection')
        folderf = os.path.join(folderr, 'captures')
        foldertm = os.path.join(folderr, 'trackedMolecule')
        folderp = os.path.join(folderr, 'peaks')
        folderpl = os.path.join(folderr, 'plots')
        if os.path.exists(folderb) == False:
            os.mkdir(folderb)
            os.mkdir(foldero)
            os.mkdir(folderf)
            os.mkdir(foldertm)
            os.mkdir(folderp)

In [8]:
def findBackground(img):
    avgz_fit = np.zeros(img.shape, np.float64)
    len_x = img.shape[0]
    len_y = img.shape[1]
    z = img.flatten()
    z = z.reshape((len(z),1))
    y = np.array([ i for i in range(len_y) ]*len_x)
    x = np.array([ [i]*len_y for i in range(len_x)  ])
    x = x.flatten()
    xx = np.power(x, 2)
    yy = np.power(y, 2)
    xy = x*y
    ones = np.ones(len(z))
    H = np.array([ xx, yy, xy, x, y, ones ])
    H = H.T
    c = np.dot(H.T, H)
    c = np.linalg.inv(c) 
    c = np.dot(c, H.T)
    c = np.dot(c, z)
    z_fit = np.dot(H, c)
    z_fit = z_fit.reshape((len_x, len_y)).astype(np.uint16)
    return(z_fit)

In [9]:
def findObjects(img, thrval):
    mean = np.mean(img)
    saliency = np.power(img - mean,2)
    dx = ndimage.sobel(saliency, 0)  # horizontal derivative
    dy = ndimage.sobel(saliency, 1)  # vertical derivative
    mag = np.hypot(dx, dy)  # magnitude
    mag *= 255 / np.max(mag)  # normalize
    binary_mask = mag >= thrval
    clean_mask = ndimage.morphology.binary_fill_holes(binary_mask).astype(np.uint8)
    return(clean_mask,mag.astype(np.uint8),binary_mask.astype(np.uint8))

In [10]:
def describeContour(img,thr,point):
    contours = cv2.findContours(thr,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)
    lowdis = -999
    for icont in contours[0]:
        bdis = cv2.pointPolygonTest(icont,point, True)
        if bdis > lowdis:
            lowdis = bdis
            cont = icont
    if lowdis == -999:
        return(False)
    area = cv2.contourArea(cont)
    per = cv2.arcLength(cont, True)
    M = cv2.moments(cont)
    x = (int(M['m10']/M['m00']) if M['m00'] > 0 else 0)
    y = (int(M['m01']/M['m00']) if M['m00'] > 0 else 0)
    mask = np.zeros(img.shape,np.uint8)
    cv2.drawContours(mask, [cont], -1, 255,-1)
    min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(img, mask)
    mean = cv2.mean(img, mask)[0]
    return([cont,x,y,area,per,mean,max_loc[1],max_loc[0],max_val,mask])

In [11]:
def clickPoint(event, x, y, flags, param):
    global refPt
    if event == cv2.EVENT_LBUTTONDOWN:
        refPt = [y,x]

In [12]:
def onFrame(imgnum, img):
    global points
    global refPt
    global stackedPoints
    eimg = my_convert_16U_2_8U(img)
    pct = [x[imgnum][1] for x in  stackedPoints]
    for p in pct:
        cv2.circle(eimg, (p[1],p[0]), 7, (255,255,255),-1)
    while True:
        refPt = []
        cv2.imshow("image", eimg)
        cv2.setMouseCallback("image", clickPoint)
        k = cv2.waitKey(0)
        cv2.destroyAllWindows()
        if (k==esc or k==left or k==right or k==enter):
            if k==enter and refPt != []:
                points.append([imgnum, refPt])
            break
    return(k)

In [13]:
def setGain(img):
    ratios = [1,1.5,2,4,8,12,16,20,24,100]
    n = 0
    while True:
        r = ratios[n]
        maxm = int(65535/r)
        dimg = img.copy()
        dimg[dimg>maxm]=65535
        if np.amin(dimg) >= maxm:
            break
        eimg = my_convert_16U_2_8U(dimg*r)
        cv2.imshow("image", eimg)
        cv2.setMouseCallback("image", clickPoint)
        k = cv2.waitKey(0)
        cv2.destroyAllWindows()
        if k==enter or n == len(ratios)-1:
            n+=1
        elif k==esc:
            break
    return(ratios[n],eimg)

In [14]:
def getLocalPeaks(img):
    gimg = ndimage.filters.gaussian_filter(img,1)
    peaks = peak_local_max(gimg,min_distance=4)
    for p in peaks:
        gimg[p[0],p[1]] = 0
    #io.imshow(gimg)
    return([peaks,gimg])

In [23]:
wantGain=True

'''Ask for picture location'''
#print("\n please choose image!")
#dirtouse = askdirectory(title = "Select image", initialdir = '/home/edwin/Desktop/')
dirtouse = 'C:/Images/2023/3-20-2023 5 mM spd condensation focus by hand/positions'
print(dirtouse)
primary_dir = sorted([os.path.join(dirtouse,x) for x in os.listdir(dirtouse)])

setanalysisloc = os.path.join(dirtouse,'Analysis.csv')
setanalysis = pd.DataFrame()
for set in primary_dir:
    print(set)
    imset = os.listdir(set)
    first = float(imset[0][:-4].replace('_','.'))
    pointcounter = 0
    frameanalysisloc = os.path.join(dirtouse,os.path.join(set,'Analysis.csv'))
    frameanalysis = pd.DataFrame()
    while True:
        imgset = []
        gbgdImgset = []
        points = []
        timeset = []
        pointcounter+=1
        subfolder = os.path.join(set,'point_'+str(pointcounter))
        if os.path.exists(subfolder):
            break 

        for imgname in imset:
            pictouse = os.path.join(set,imgname)
            imgname = imgname[:-4]
            time = float(imgname.replace('_','.')) # - first
            timeset.append(time)
            img = io.imread(pictouse,-1)
            imgset.append(img)
            bgd = findBackground(img).astype(np.uint16)
            bgdImg = cv2.subtract(img, bgd)
            gbgdImg = ndimage.filters.gaussian_filter(bgdImg,1)
            gbgdImgset.append(gbgdImg)

            #This part is for picking molecules
            if wantGain == True:
                plow, phigh = np.percentile(img, (12, 98))
                eimg = exposure.rescale_intensity(img, in_range=(plow, phigh))
            refPt = []
            cv2.imshow("image", eimg)
            cv2.setMouseCallback("image", clickPoint)
            k = cv2.waitKey(0)
            cv2.destroyAllWindows()
            if k==enter and refPt != []:
                points.append(refPt)
            elif k==esc:
                points.append(None)

        #Analyzes molecules
        bbset = []
        sobelset = []
        pointset = []
        threshset = []
        peakset = []

        pdata = pd.DataFrame()
        for n in range(len(points)):
            point = points[n]
            if point == None:
                continue
            img = imgset[n]
            gbgdImg = gbgdImgset[n]
            time = timeset[n]

            #find centroid and objects to save
            brightBox=getBrightBox(img,point,outL)
            gbgdbBox=getBrightBox(gbgdImg,point,outL)
            foundobjsn,edges,foundobjs = findObjects(gbgdbBox[0],10)
            cont = describeContour(brightBox[0],foundobjsn,(point[1]-brightBox[1],point[0]-brightBox[2]))
            outMean = cv2.mean(brightBox[0],mask=(255 - cont[9]))[0]
            p = [int(cont[2])+brightBox[2],int(cont[1])+brightBox[1]]

            #save files
            cv2.circle(eimg, (p[1],p[0]), 20, (255,255,255),2)
            miss = brightBox[4]  
            bboo = cv2.copyMakeBorder(brightBox[0], miss[0], miss[1], miss[2], miss[3], cv2.BORDER_CONSTANT)
            ftoo = cv2.copyMakeBorder(cont[-1], miss[0], miss[1], miss[2], miss[3], cv2.BORDER_CONSTANT)
            otoo = cv2.copyMakeBorder(edges, miss[0], miss[1], miss[2], miss[3], cv2.BORDER_CONSTANT)
            masked = bboo*ftoo
            peaks, peoo = getLocalPeaks(masked)
            lAthr = measure.label(ftoo)
            props = measure.regionprops_table(lAthr,intensity_image=bboo,
                                            properties=['area','bbox', 'bbox_area', 'centroid', 
                                                        'convex_area', 'eccentricity', 'equivalent_diameter', 
                                                        'inertia_tensor', 'inertia_tensor_eigvals', 
                                                        'label', 'local_centroid', 'major_axis_length', 
                                                        'max_intensity', 'mean_intensity', 
                                                        'min_intensity', 'minor_axis_length', 'moments', 
                                                        'moments_central', 'moments_hu', 
                                                        'moments_normalized', 'orientation', 'perimeter', 
                                                        'solidity', 'weighted_centroid', 
                                                        'weighted_local_centroid', 'weighted_moments', 
                                                        'weighted_moments_central', 'weighted_moments_hu', 
                                                        'weighted_moments_normalized'])
            props['x'] = p[0]
            props['y'] = p[1]
            props['time']=str(time)
            props['npeaks'] = len(peaks)
            pdata = pdata.append(pd.DataFrame(props).dropna(axis=1))
            bbooName = os.path.join(subfolder,('brightBox_'+str(n)+'.tif'))
            otooName = os.path.join(subfolder,('edges_'+str(n)+'.tif'))
            ftooName = os.path.join(subfolder,('capture_'+str(n)+'.tif'))
            eimgName = os.path.join(subfolder,('point_'+str(n)+'.tif'))
            pimgName = os.path.join(subfolder,('peak_'+str(n)+'.tif'))
            bbset.append([bbooName,bboo])
            sobelset.append([otooName,otoo])
            threshset.append([ftooName,ftoo])
            pointset.append([eimgName,eimg])
            peakset.append([pimgName,peoo])
        savef = True
        for i, pair in enumerate(threshset):
            cv2.imshow("image", pair[1])
            k = cv2.waitKey(0)
            cv2.destroyAllWindows()
            if (k==esc or k==left or k==right or k==enter):
                if k==enter:
                    continue
                elif k==esc:
                    savef = False
                    break
        if savef == True:
            os.mkdir(subfolder)
            if len(frameanalysis) == 0:
                frameanalysis = pdata.copy()
            else:
                pd.concat([frameanalysis,pdata])
            pdata.to_csv(os.path.join(subfolder,'Analysis.csv'))
            for n in range(len(bbset)):
                cv2.imwrite(bbset[n][0],bbset[n][1])
                cv2.imwrite(sobelset[n][0],sobelset[n][1])
                cv2.imwrite(threshset[n][0],threshset[n][1])
                cv2.imwrite(pointset[n][0],pointset[n][1])
                cv2.imwrite(peakset[n][0],peakset[n][1])
        go = input("Enter 'E' to exit, anything else to do another molecule")
        if go == 'E':
            break
    if len(frameanalysis) > 0:
        frameanalysis.to_csv(os.path.join(frameanalysisloc,'Analysis.csv'))
        if len(setanalysis) == 0:
            setanalysis = frameanalysis.copy()
        else:
            pd.concat([setanalysis,frameanalysis])
    break
setanalysis.to_csv(os.path.join(setanalysisloc,'Analysis.csv'))

C:/Images/2023/3-20-2023 5 mM spd condensation focus by hand/positions
C:/Images/2023/3-20-2023 5 mM spd condensation focus by hand/positions\0


OSError: Cannot save file into a non-existent directory: 'C:\Images\2023\3-20-2023 5 mM spd condensation focus by hand\positions\0\Analysis.csv'

In [16]:
'''Ask for picture location'''
#print("\n please choose image!")
#dirtouse = askdirectory(title = "Select image", initialdir = '/home/edwin/Desktop/')
dirtouse = 'C:/Images/2023/3-8-2023 Condensing T4 focus by hand/0.5 mM final/positions'
primary_dir = sorted(os.listdir(dirtouse))
counter = 0 

sbigloc = os.path.join(dirtouse,'Analysis.csv')
sbiganal = []
for setch in primary_dir:
    if setch=='Analysis.csv':
        continue
    set = os.path.join(dirtouse,setch)
    imgset = []
    #for pro in os.listdir(set):
    #    if pro[-4:] == '.tif':
    #        imgset.append(float(pro[:-4].replace('_','.')))
    #first = min(imgset)
    #print(first)
    biganal = []
    for pro in os.listdir(set):
        prog = os.path.join(set,pro)
        if os.path.isdir(prog) == True:
            anpath = os.path.join(prog,'Analysis.csv')
            with open(anpath,'r') as f:
                s = csv.reader(f)
                slo = []
                fi = 'erg'
                counter+=1
                for row in s:
                    slo.append(row+[str(setch)+'_'+str(pro)])
                #    if fi == True:
                #        area = row[8]
                #        row[-2] = float(row[-2]) + first
                #        fi = False
                #        slo.append(row+[str(float(row[8])/float(area)),str(setch)+'_'+str(pro)])
                #        #counter+=1
                #    elif fi == 'erg':
                #        fi = True
                #    else:
                #        row[-2] = float(row[-2]) + first
                #        slo.append(row+[str(float(row[8])/float(area)),str(setch)+'_'+str(pro)])
                #        #counter+=1
                biganal.extend(slo)
    bigloc = os.path.join(set,'Analysis.csv')
    with open(bigloc,'w') as f:
        wre = csv.writer(f)
        wre.writerows(biganal)
    sbiganal.extend(biganal)
with open(sbigloc,'w') as f:
    wre = csv.writer(f)
    wre.writerows(sbiganal)
print(counter)

0
